In [1]:
import pandas as pd
import numpy as np

In [2]:
from selenium import webdriver
from selenium.webdriver import ActionChains  # 액션체인 활성화
import time
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import requests
from bs4 import BeautifulSoup



In [5]:
from selenium import webdriver
from selenium.webdriver import ActionChains  # 액션체인 활성화
import time
from selenium.webdriver.common.keys import Keys

dr = webdriver.Chrome('/Users/hayeon/Downloads/chromedriver_mac_arm64/chromedriver')
#dr.get('https://search.shopping.naver.com/search/all?origQuery=%EC%A0%84%ED%86%B5%EC%A3%BC&pagingIndex=1&pagingSize=80&productSet=total&query=%EC%A0%84%ED%86%B5%EC%A3%BC&sort=review&timestamp=&viewType=list')


all_data = []
name = []
spec = []
category = []
price = []
star = []
link = []


df = pd.DataFrame(columns=['name', 'category', 'spec', 'price', 'star', 'review_num', 'click', 'regist', 'link'])

for i in range(1, 4):
    date = []
    date2 = []
    rev_n = []
    rev_n2 = []
    
    print(i)
    
    dr.get('https://search.shopping.naver.com/search/category/100010442?catId=50014520&frm=NVSHMDL&origQuery&pagingIndex={0}&pagingSize=40&productSet=model&query&sort=review&timestamp=&viewType=list'.format(i))

    
    scroll_location = dr.execute_script("return document.body.scrollHeight")
    
    while True:
        #현재 스크롤의 가장 아래로 내림
        dr.execute_script("window.scrollTo(0,document.body.scrollHeight)")

        #전체 스크롤이 늘어날 때까지 대기
        time.sleep(5)

        #늘어난 스크롤 높이
        scroll_height = dr.execute_script("return document.body.scrollHeight")
        #time.sleep(3)

        #늘어난 스크롤 위치와 이동 전 위치 같으면(더 이상 스크롤이 늘어나지 않으면) 종료
        if scroll_location == scroll_height:
            break

        #같지 않으면 스크롤 위치 값을 수정하여 같아질 때까지 반복
        else:
            #스크롤 위치값을 수정
            scroll_location = dr.execute_script("return document.body.scrollHeight")
            

    time.sleep(10)

    
    mydata_name = dr.find_elements_by_css_selector('div.basicList_title__VfX3c') # 상품명
    #time.sleep(4)
    mydata_category = dr.find_elements_by_css_selector('div.basicList_depth__SbZWF') # 카테고리
    mydata_spec = dr.find_elements_by_css_selector('div.basicList_detail_box__OoXKt') # 용량,도수,용기타입
    mydata_price = dr.find_elements_by_css_selector('span.price_num__S2p_v') # 가격
    mydata_star = dr.find_elements_by_css_selector('span.basicList_star__UzKiv') # 별점
    mydata_date = dr.find_elements_by_css_selector('span.basicList_etc__LSkN_') # 등록일자
    review_num = dr.find_elements_by_css_selector('em.basicList_num__sfz3h') # 리뷰 개수
    mydata_link = dr.find_elements_by_css_selector('div.basicList_title__VfX3c > a') # 링크
    
    for data_d in mydata_date:
        date.append(data_d.text)
        
    for i in range(0, len(date), 3):
        date2.append(date[i:i+3])
        
    for data_rn in review_num:
        rev_n.append(data_rn.text)
        
    for i in range(0, len(rev_n), 2):
        rev_n2.append(rev_n[i:i+2])
    
    for data_n, data_c, data_s, data_p, data_star, data_rn, data_d, data_link in zip(mydata_name, mydata_category, mydata_spec, mydata_price, mydata_star, rev_n2, date2, mydata_link):#, mydata_spec):
        #print(data_n.text, '/', data_c.text, '/', data_s.text, '/', data_p.text, '/', data_star.text, '/', data_rn[0], '/', data_rn[1], '/', data_d[0], '/', data_link.get_attribute('href'))
        # 상품명, 카테고리, 용량/도수/용기타입 등, 가격, 별점, 리뷰 개수, 등록일
        df_li = [data_n.text, data_c.text, data_s.text, data_p.text, data_star.text, data_rn[0], data_rn[1], data_d[0], data_link.get_attribute('href')]

        df = df.append(pd.Series(df_li, index=df.columns), ignore_index=True)
        
        name.append(data_n.text)
        category.append(data_c.text)
        spec.append(data_s.text)
        price.append(data_p.text)
        star.append(data_star.text)
        link.append(data_link)
        #rev_n.append(data_rn.text)
        #date.append(data_d.text)
    
    
    #print(df)

    

1
2
3


In [10]:
for i in range(0, len(df)):
    df['review_num'].iloc[i] = df['review_num'].iloc[i].replace(',', '')

In [14]:
df['review_num'] = df['review_num'].astype('int')

In [15]:
df

,name,category,spec,price,star,review_num,click,regist,link
0,배상면주가 느린마을 막걸리 6도 750ml,식품주류전통주막걸리/탁주,용량 : 750ml도수 : 6%용기타입 : 페트병,"3,600원",별점 4.8,34247,156,등록일 2022.06.,https://cr.shopping.naver.com/adcr.nhn?x=hwOEt...
1,복순도가 손막걸리 6.5도 935ml,식품주류전통주막걸리/탁주,용량 : 935ml도수 : 6.5%용기타입 : 페트병,"12,000원",별점 4.8,25404,212,등록일 2020.10.,https://cr.shopping.naver.com/adcr.nhn?x=dVVid...
2,한강주조 표문 막걸리 6도 500ml,식품주류전통주막걸리/탁주,용량 : 500ml도수 : 6%용기타입 : 페트병,"3,500원",별점 4.7,12504,35,등록일 2021.10.,https://cr.shopping.naver.com/adcr.nhn?x=PbTTS...
3,더한 서울의밤 25도 375ml,식품주류전통주리큐르주,용량 : 375ml도수 : 25%용기타입 : 유리병,"7,900원",별점 4.9,8152,55,등록일 2020.09.,https://cr.shopping.naver.com/adcr.nhn?x=PNOzq...
4,배상면주가 빙탄복 7도 370ml,식품주류전통주국산와인/과일주,도수 : 7%용량 : 370ml용기타입 : 유리병,"28,620원",별점 4.8,7723,92,등록일 2017.07.,https://cr.shopping.naver.com/adcr.nhn?x=nTrgc...
...,...,...,...,...,...,...,...,...,...
115,댄싱사이더 애플사이더 댄싱파파 7도 330ml,식품주류전통주국산와인/과일주,종류 : 스파클링당도 : 드라이목넘김 : 부드러움도수 : 7%용량 : 330ml용기...,"5,900원",별점 4.8,413,21,등록일 2022.06.,https://cr.shopping.naver.com/adcr.nhn?x=Eyfr9...
116,배상면주가 느린마을 소주 21도 375ml,식품주류전통주소주,용량 : 375ml도수 : 21%용기타입 : 유리병,"11,880원",별점 4.8,412,53,등록일 2022.06.,https://cr.shopping.naver.com/adcr.nhn?x=PlY3S...
117,민속주 안동소주 45도 800ml,식품주류전통주소주,용량 : 800ml도수 : 45%용기타입 : 도자기병,"40,840원",별점 4.9,411,25,등록일 2021.09.,https://cr.shopping.naver.com/adcr.nhn?x=tBYqW...
118,중원양조 사랑할때 사과주 20도 300ml,식품주류전통주국산와인/과일주,도수 : 20%용량 : 300ml용기타입 : 유리병아로마 : 사과,"4,430원",별점 4.9,410,25,등록일 2022.05.,https://cr.shopping.naver.com/adcr.nhn?x=T8%2F...


In [16]:
df.to_csv('./list_new.csv', encoding='utf-8', inde)